In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pyarrow as pa
import pyarrow.parquet as pq
import datetime 
import importlib
import utils
import torch
import ast
# import wrds

from transformers import DistilBertTokenizer, DistilBertModel, \
BertTokenizer, BertModel, LongformerTokenizer, LongformerModel

importlib.reload(utils)

from utils import *

# Loading of useful macroeconomic information

In this part, we load macroeconomic information from WRDS through some SQL queries. This way, we will end up with the file 'crsp_monthly.csv' (in data folder), containing in particular all companies'market capitalizations at the time of their earnings calls.

In [ ]:
start_date = '2009-01-01'
end_date= '2020-12-31'

In [ ]:
# Connection to wrds: to run this, a WRDS account is needed
conn = wrds.Connection()

In [ ]:
### Get S&P500 Index Membership from CRSP
###  monthly frequency of the data, 

sp500 = conn.raw_sql("""
                        select a.*, b.date, b.ret
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='02/15/2005' 
                        order by date;
                        """, date_cols=['start', 'ending', 'date']) 

# Save data
sp500.to_csv('sp500.csv', index=False)

In [ ]:
### Add Other Company Identifiers from CRSP.MSENAMES
### - You don't need this step if only PERMNO is required
### - This step aims to add TICKER, SHRCD, EXCHCD and etc. 
# chiedere

mse = conn.raw_sql("""
                        select comnam, ncusip, namedt, nameendt, 
                        permno, shrcd, exchcd, hsiccd, ticker
                        from crsp.msenames
                   
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))


# Save data
mse.to_csv('mse.csv', index=False)

In [ ]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

# Save data
sp500_full.to_csv('sp500_full.csv', index=False)

In [ ]:
### Add Compustat Identifiers
### - Link with Compustat's GVKEY and IID if need to work with 
###   fundamental data
### - Linkage is done through crsp.ccmxpf_linktable

ccm=conn.raw_sql("""
                  select gvkey, liid as iid, lpermno as permno,
                  linktype, linkprim, linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """, date_cols=['linkdt', 'linkenddt'])

# if linkenddt is missing then set to today date
ccm['linkenddt']=ccm['linkenddt'].fillna(pd.to_datetime('today'))

# Save data
ccm.to_csv('ccm.csv', index=False)

In [ ]:
# Merge the CCM data with S&P500 data
# First just link by matching PERMNO
sp500ccm = pd.merge(sp500_full, ccm, how='left', on=['permno'])

# Then set link date bounds
sp500ccm = sp500ccm.loc[(sp500ccm['date']>=sp500ccm['linkdt'])\
                        &(sp500ccm['date']<=sp500ccm['linkenddt'])]

# Rearrange columns for final output
sp500ccm = sp500ccm.drop(columns=['namedt', 'nameendt', 'linktype', \
                                  'linkprim', 'linkdt', 'linkenddt'])
sp500ccm = sp500ccm[['date', 'permno', 'comnam', 'ncusip',\
                     'shrcd', 'exchcd', 'hsiccd', 'ticker', \
                     'gvkey', 'iid', 'start', 'ending', 'ret']]

# Save data
sp500ccm.to_csv('sp500ccm.csv', index=False)

In [ ]:
### Add CIKs and Link with SEC Index Files using CIK
names = conn.raw_sql(""" select gvkey, cik, sic from comp.names """)

# Merge sp500 constituents table with names table
sp500cik = pd.merge(sp500ccm, names, on='gvkey',  how='left')

# Save data
sp500cik.to_csv('sp500cik.csv', index=False)

In [ ]:
#extracting permno and gvkey to use as identifiers for the next steps
per_gev= sp500cik[['permno', 'gvkey']].drop_duplicates()
# select the relvant permno to extract only the information about the companies in the sp500
selected_permno = sp500cik['permno'].unique()

In [ ]:
# SQL Query
crsp_monthly= conn.raw_sql(f"""
    SELECT msf.permno,  msf.mthcaldt AS date,
           msf.mthret AS ret, msf.shrout, msf.mthprc AS altprc,
           ssih.primaryexch, ssih.siccd
    FROM crsp.msf_v2 AS msf
    INNER JOIN crsp.stksecurityinfohist AS ssih
    ON msf.permno = ssih.permno
    AND ssih.secinfostartdt <= msf.mthcaldt
    AND msf.mthcaldt <= ssih.secinfoenddt
    WHERE msf.mthcaldt BETWEEN '{start_date}' AND '{end_date}'
    and msf.permno in {tuple(selected_permno)}
    AND ssih.sharetype = 'NS'
    AND ssih.securitytype = 'EQTY'
    AND ssih.securitysubtype = 'COM'
    AND ssih.usincflg = 'Y'
    AND ssih.issuertype IN ('ACOR', 'CORP')
    AND ssih.primaryexch IN ('N', 'A', 'Q')
    AND ssih.conditionaltype IN ('RW', 'NW')
    AND ssih.tradingstatusflg = 'A';
""")

# create a new column for market capitalization
crsp_monthly = crsp_monthly.assign(mktcap=crsp_monthly["shrout"] * crsp_monthly["altprc"] * 1000)
crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])
crsp_monthly=crsp_monthly.sort_values(by=['date'], ascending=True)
crsp_monthly['date'] = pd.to_datetime(crsp_monthly['date'])

# Create lagged marketcap
mktcap_lag = (
   crsp_monthly
    .assign(
        date=lambda x: x["date"] + pd.DateOffset(months=1),
        mktcap_lag=lambda x: x["mktcap"]
    )
    .get(["permno", "date", "mktcap_lag"])
)
crsp_monthly= pd.merge(crsp_monthly, mktcap_lag, how="left", on=["permno", "date"]) 
crsp_monthly.dropna(subset=['mktcap_lag'], inplace=True)
crsp_monthly=crsp_monthly.reset_index(drop=True)
crsp_monthly = crsp_monthly.merge(per_gev, on='permno', how='left')

# Save data
crsp_monthly.to_csv('crsp_monthly_correct.csv', index=False)

# Dataset creation

Now we can proceed with the creation of the final datasets that we will use to run the models. Specifically, the required tasks are:
- Tokenize all available texts using BERT, DistilBERT and Longformer tokenizers;
- From the tokenized texts, create their embeddings through the corresponding models (resp. BERT, DistilBERT and Longformer);
- Extract only CLS token

In [ ]:
# Load useful dataset
dataset = pd.read_parquet('data_4ml.parquet')
crsp = pd.read_csv('crsp_monthly_correct.csv')

In [ ]:
# Add gvkey to dataset
gvkey_500 = [str(num) for num in crsp['gvkey']]
dataset['gvkey'] = dataset['gvkey'].apply(lambda x: x.lstrip("0"))
dataset = dataset[dataset['gvkey'].isin(gvkey_500)]

# Group by 'transcriptid' and concatenate 'componenttext' values, handling None values
# create one unique row for each transcript id
dataset = (
    dataset.groupby('transcriptid', as_index=False)
    .agg({'componenttext': lambda x: ' '.join(filter(None, x)), 'mostimportantdateutc': 'first', 'gvkey': 'first'}) 
)

# Deal with time
dataset['mostimportantdateutc'] = pd.to_datetime(dataset['mostimportantdateutc'])
dataset['month'] = dataset['mostimportantdateutc'].dt.month
dataset['year'] = dataset['mostimportantdateutc'].dt.year
dataset.drop(columns=['mostimportantdateutc'], inplace=True)

# Save results
dataset.to_csv('data/dataset_with_texts.csv')

## 1) DistilBERT Dataset

In [ ]:
# Tokenization Phase ...

texts = pd.read_csv("data/dataset_with_texts.csv")
texts_to_tokenize = [row ['componenttext'] for _, row in texts.iterrows()]
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenized_texts = tokenizer(texts_to_tokenize, max_length=512, 
                            padding="max_length", truncation=True, return_tensors='pt')

ids_list = []
for i in range(len(tokenized_texts['input_ids'])):
    ids_list.append(tokenized_texts['input_ids'][i].tolist())

mask_list = []
for i in range(len(tokenized_texts['attention_mask'])):
    mask_list.append(tokenized_texts['attention_mask'][i].tolist())

texts['tokenized_text'] = ids_list
texts['attention_mask'] = mask_list

texts.to_csv('data/embeddings_distilbert_correct.csv')

In [ ]:
# Create embeddings ...

data = pd.read_csv('data/embeddings_distilbert_correct.csv')

distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased").to('cuda')

def get_embedding(row):
    tok_text_list = [ast.literal_eval(row['tokenized_text'])]
    att_list = [ast.literal_eval(row['attention_mask'])]
    texts_tok = torch.tensor(tok_text_list, device='cuda')
    texts_att = torch.tensor(att_list, device='cuda')
    with torch.no_grad():
        embedding = distilbert(input_ids=texts_tok, attention_mask=texts_att)
    hidden_state = embedding.last_hidden_state[:, 0, :] 
    hidden_state = hidden_state.to('cpu')
    return hidden_state.detach().numpy()

chunk_size = 100

for start in range(0, len(dataset), chunk_size):
    try:
        end = start + chunk_size
        chunk = dataset.iloc[start:end] 
        dataset.loc[start:end-1, 'embeddings'] = chunk.apply(get_embedding, axis=1)
        # ds[['embeddings']].to_csv('last_results_bert_correct.csv')
        print(f'Chunk {start}-{end} done')
    except:
        print(f"Could not retrieve embedding for chunk {start}-{end}")

dataset.to_csv('data/embeddings_distilbert_correct.csv')

## 2) BERT Dataset

In [ ]:
# Tokenization Phase ...

texts = pd.read_csv("data/dataset_with_texts.csv")
texts_to_tokenize = [row ['componenttext'] for _, row in texts.iterrows()]
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenized_texts = tokenizer(texts_to_tokenize, max_length=512, 
                            padding="max_length", truncation=True, return_tensors='pt')

ids_list = []
for i in range(len(tokenized_texts['input_ids'])):
    ids_list.append(tokenized_texts['input_ids'][i].tolist())

mask_list = []
for i in range(len(tokenized_texts['attention_mask'])):
    mask_list.append(tokenized_texts['attention_mask'][i].tolist())

texts['tokenized_text'] = ids_list
texts['attention_mask'] = mask_list

texts.to_csv('data/embeddings_bert_correct.csv')

In [ ]:
# Create embeddings ...

data = pd.read_csv('data/embeddings_bert_correct.csv')

bert = BertModel.from_pretrained("bert-base-uncased").to('cuda')

def get_embedding(row):
    tok_text_list = [ast.literal_eval(row['tokenized_text'])]
    att_list = [ast.literal_eval(row['attention_mask'])]
    texts_tok = torch.tensor(tok_text_list, device='cuda')
    texts_att = torch.tensor(att_list, device='cuda')
    with torch.no_grad():
        embedding = bert(input_ids=texts_tok, attention_mask=texts_att)
    hidden_state = embedding.last_hidden_state[:, 0, :] 
    hidden_state = hidden_state.to('cpu')
    return hidden_state.detach().numpy()

chunk_size = 100

for start in range(0, len(dataset), chunk_size):
    try:
        end = start + chunk_size
        chunk = dataset.iloc[start:end] 
        dataset.loc[start:end-1, 'embeddings'] = chunk.apply(get_embedding, axis=1)
        # ds[['embeddings']].to_csv('last_results_bert_correct.csv')
        print(f'Chunk {start}-{end} done')
    except:
        print(f"Could not retrieve embedding for chunk {start}-{end}")

dataset.to_csv('data/embeddings_bert_correct.csv')

## 3) Longformer Dataset

In [ ]:
# Tokenization Phase ...

texts = pd.read_csv("data/dataset_with_texts.csv")
texts_to_tokenize = [row ['componenttext'] for _, row in texts.iterrows()]
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
tokenized_texts = tokenizer(texts_to_tokenize, max_length=4096, 
                            padding="max_length", truncation=True, return_tensors='pt')

ids_list = []
for i in range(len(tokenized_texts['input_ids'])):
    ids_list.append(tokenized_texts['input_ids'][i].tolist())

mask_list = []
for i in range(len(tokenized_texts['attention_mask'])):
    mask_list.append(tokenized_texts['attention_mask'][i].tolist())

texts['tokenized_text'] = ids_list
texts['attention_mask'] = mask_list

texts.to_csv('data/embeddings_longformer_correct.csv')

In [ ]:
# Create embeddings ...

data = pd.read_csv('data/embeddings_longformer_correct.csv')

longformer = LongformerModel.from_pretrained('allenai/longformer-base-4096').to('cuda')

def get_embedding(row):
    tok_text_list = [ast.literal_eval(row['tokenized_text'])]
    att_list = [ast.literal_eval(row['attention_mask'])]
    texts_tok = torch.tensor(tok_text_list, device='cuda')
    texts_att = torch.tensor(att_list, device='cuda')
    with torch.no_grad():
        embedding = longformer(input_ids=texts_tok, attention_mask=texts_att)
    hidden_state = embedding.last_hidden_state[:, 0, :] 
    hidden_state = hidden_state.to('cpu')
    return hidden_state.detach().numpy()

chunk_size = 100

for start in range(0, len(dataset), chunk_size):
    try:
        end = start + chunk_size
        chunk = dataset.iloc[start:end] 
        dataset.loc[start:end-1, 'embeddings'] = chunk.apply(get_embedding, axis=1)
        # ds[['embeddings']].to_csv('last_results_bert_correct.csv')
        print(f'Chunk {start}-{end} done')
    except:
        print(f"Could not retrieve embedding for chunk {start}-{end}")

dataset.to_csv('data/embeddings_longformer_correct.csv')